In [9]:
import pandas as pd
import os

%matplotlib inline

In [2]:
def apply_labels(df,table_type):
    '''
    Replace values with human readable lablels.
    '''
    df_label = labels[labels['table'] == table_type]
    for field in df_label['field'].unique():
        newdf = df_label[df_label['field'] == field]
        local_series = pd.Series(newdf['text'].values, index=newdf['value'])
        df[field] = df[field].map(local_series)
    
    return df

In [10]:
# Daysim data
trip = pd.read_csv(r'../../../outputs/daysim/_trip.tsv', sep='\t')
person = pd.read_csv(r'../../../outputs/daysim/_person.tsv', sep='\t')
hh = pd.read_csv(r'../../../outputs/daysim/_household.tsv', sep='\t')

# Add labels
labels = pd.read_csv(r'../../../scripts/summarize/inputs/calibration/variable_labels.csv')

trip = apply_labels(trip,'Trip')
person = apply_labels(person,'Person')
hh = apply_labels(hh,'Household')

In [4]:

special_taz = pd.read_csv(r'../../../scripts/summarize/inputs/special_needs_taz.csv')
rgc_taz = pd.read_csv(r'../../../scripts/summarize/inputs/rgc_taz.csv')

In [5]:
# Join RGC, low income, and minority data, based on home location

trip_hh = pd.merge(trip,hh,on='hhno',how='left')

trip_hh = pd.merge(trip_hh, special_taz, left_on='hhtaz', right_on='TAZ', how='left')
trip_hh = pd.merge(trip_hh, rgc_taz, left_on='hhtaz', right_on='taz', how='left')

In [6]:
trip_hh = trip_hh[trip_hh['mode']!='School Bus']

In [7]:
pd.options.display.float_format = '{:.2f}'.format

## Mode Share
Based on home location

In [20]:
def mode_share(trip_hh):
    """
    Produce trip mode share for low income, minority, and regional populations
    """
    # Regional total
    df_reg = pd.DataFrame(
        trip_hh.groupby('mode').sum()['trexpfac']/trip_hh['trexpfac'].sum())
    df_reg.columns = ['Region']
    df_reg = df_reg.reset_index()
    
    # Low Income
    df = trip_hh[['trexpfac','Low Income','mode']].groupby(['Low Income','mode']).sum().reset_index()
    tot_trips = pd.DataFrame(df.groupby('Low Income').sum()['trexpfac']).reset_index()
    df = pd.merge(df,tot_trips,on='Low Income', suffixes=['_mode','_total'])
    df['mode_share'] = (df['trexpfac_mode']/df['trexpfac_total'])

    df_inc = df.pivot(index='Low Income', columns='mode', values='mode_share')
    df_inc.index = ['Other','People of Lower Income']
    
    # Minority
    df = trip_hh[['trexpfac','Minority','mode']].groupby(['Minority','mode']).sum().reset_index()
    tot_trips = pd.DataFrame(df.groupby('Minority').sum()['trexpfac']).reset_index()
    df = pd.merge(df,tot_trips,on='Minority', suffixes=['_mode','_total'])
    df['mode_share'] = (df['trexpfac_mode']/df['trexpfac_total'])

    df_race = df.pivot(index='Minority', columns='mode', values='mode_share')
    df_race.index = ['Other','People of Color']
    
    # Merge all dataframes
    df = pd.merge(pd.DataFrame(df_race.loc['People of Color']).reset_index(),
         pd.DataFrame(df_inc.loc['People of Lower Income']).reset_index())
    df = pd.merge(df_reg, df)
    df.index = df['mode']
    df = df.drop('mode', axis=1)
    return df

### All Trips

In [21]:
pd.options.display.float_format = '{:.2f}%'.format
df = mode_share(trip_hh)
df

### Commute Trips

In [22]:
commute_trips = trip_hh[trip_hh['dpurp'] == 'Work']
commute_trips = commute_trips[commute_trips['mode'] != 'School Bus']
df = mode_share(commute_trips)
df

### Access to Frequent Transit

In [23]:
df = pd.read_csv(r'../../../outputs/transit/freq_transit_access.csv')
# Join parcel data (transit access) to household recods
df = pd.merge(hh[['hhparcel','hhsize','hhtaz']],df[['PARCELID','dist_frequent']],how='left',left_on='hhparcel',right_on='PARCELID')

# join to geography fiels
df = pd.merge(df, special_taz, left_on='hhtaz', right_on='TAZ', how='left')
df = pd.merge(df, rgc_taz, left_on='hhtaz', right_on='taz', how='left')

In [24]:
# Only include househodls on parcels within 1/2 mile of frequent transit
max_dist = 0.5
df_freq = df[df['dist_frequent'] <= max_dist]

In [25]:
# Calculate percent of people within that range

In [26]:
df_lowinc = pd.DataFrame(df_freq.groupby('Low Income').sum()['hhsize']/df.groupby('Low Income').sum()['hhsize'])

In [27]:
df_minority = pd.DataFrame(df_freq.groupby('Minority').sum()['hhsize']/df.groupby('Minority').sum()['hhsize'])

In [28]:
reg_share = df_freq.sum()['hhsize']/df.sum()['hhsize']

In [29]:
_df = pd.DataFrame([reg_share,df_minority.loc[1].values[0],df_lowinc.loc[1].values[0]])
_df.index = ['Region','People of Color','People of Low Income']
_df.columns = ['Percent with Access to Frequent Transit Service']
_df['Percent with Access to Frequent Transit Service'] = _df['Percent with Access to Frequent Transit Service']*100
_df

,Percent with Access to Frequent Transit Service
Region,36.52%
People of Color,47.86%
People of Low Income,42.29%


## Percent of People Walking and Biking for Transport

In [48]:
trip_person = pd.merge(trip,person,on=['hhno','pno'], how='left')
person_hh = pd.merge(person,hh[['hhno','hhtaz']],on='hhno',how='left')
# bike_walk_trips = trip_person[trip_person['mode'].isin(['Bike','Walk'])]
# Walk & bike modes, plus transit trips with walk access
bike_walk_trips = trip_person[trip_person['mode'].isin(['Bike','Walk']) | ((trip_person['mode'] == 'Transit') & (trip_person['dorp'] > 0))]

df = bike_walk_trips.groupby(['hhno','pno']).count()
df = df.reset_index()
df = df[['hhno','pno']]
df['bike_walk'] = True

df = pd.merge(person_hh,df,on=['hhno','pno'], how='left')
df['bike_walk'] = df['bike_walk'].fillna(False)

In [49]:
pd.options.display.float_format = '{:,.1%}'.format
reg_active = pd.DataFrame(df.groupby('bike_walk').sum()['psexpfac']/df['psexpfac'].sum()).loc[True]['psexpfac']

In [50]:
df = pd.merge(df, special_taz, left_on='hhtaz', right_on='TAZ', how='left')
df = pd.merge(df, rgc_taz, left_on='hhtaz', right_on='taz', how='left')

In [51]:
_df = (df.groupby(['Low Income','bike_walk']).sum()['psexpfac']/df['psexpfac'].sum()).reset_index()
low_inc_active = _df.ix[(_df['Low Income'] == 1) & (_df['bike_walk']==True),'psexpfac'].tolist()[0]

In [52]:
_df = (df.groupby(['Minority','bike_walk']).sum()['psexpfac']/df['psexpfac'].sum()).reset_index()
minority_active = _df.ix[(_df['Minority'] == 1) & (_df['bike_walk']==True),'psexpfac'].tolist()[0]

In [53]:
_df = pd.DataFrame([reg_active,minority_active,low_inc_active])
_df.index = ['Region','People of Color','People of Low Income']
_df.columns = ['% Walks or Bikes for Transportation']
_df.to_clipboard()

### Average Time Spent Walking

In [69]:
pd.options.display.float_format = "{0:.2f}".format
# walk_trips = trip_person[trip_person['mode'] == 'Walk']
# daily_walk_times = walk_trips.groupby(['hhno','pno']).sum()['travtime']

In [64]:
daily_tot_time = pd.DataFrame(trip_person.groupby(['pno','hhno','mode']).sum()['travtime'])
daily_tot_time = daily_tot_time.reset_index()
daily_walk_time = daily_tot_time[daily_tot_time['mode'] == 'Walk']

In [65]:
# Total walk time across region divided by total number of people
reg_walk_time = daily_walk_time['travtime'].sum()/person['psexpfac'].sum()

In [66]:
# Total for low inc and for rgc
df = pd.merge(daily_tot_time,hh[['hhno','hhtaz']],on='hhno', how='left')
df = pd.merge(df,rgc_taz,left_on='hhtaz',right_on='taz',how='left')
df = pd.merge(df,special_taz,left_on='hhtaz',right_on='TAZ',how='left')

In [67]:
low_inc_walk_time = df[df['Low Income'] == 1]['travtime'].sum()/person['psexpfac'].sum()
minority_walk_time = df[df['Minority'] == 1]['travtime'].sum()/person['psexpfac'].sum()

In [68]:
df = pd.DataFrame([reg_walk_time,minority_walk_time,low_inc_walk_time])
df.columns = ['Average Minutes Walking per Day']
df.index = ['Region','People of Color','People of Low Income']
df

,Average Minutes Walking per Day
Region,17.86
People of Color,31.89
People of Low Income,30.42


# Annual Out-of-Pocket Costs

In [60]:
annual_factor = 300
reg_costs = trip_hh[['hhno','travcost']].groupby('hhno').sum()['travcost'].mean()*annual_factor
low_inc_costs = trip_hh[trip_hh['Low Income'] == 1][['hhno','travcost']].groupby('hhno').sum()['travcost'].mean()*annual_factor
minority_costs = trip_hh[trip_hh['Minority'] == 1][['hhno','travcost']].groupby('hhno').sum()['travcost'].mean()*annual_factor

In [70]:
df = pd.DataFrame([reg_costs,minority_costs,low_inc_costs])
df.columns = ['Annual Out-of-Pocket Costs']
df.index = ['Region','People of Color','People of Low Income']
df

,Annual Out-of-Pocket Costs
Region,4289.45
People of Color,3832.45
People of Low Income,3655.29


### Daily VMT per capita

In [8]:
driver_trips = trip[(trip['dorp'] == 1) & (trip['mode'].isin(['SOV','HOV2','HOV3+']))]

In [12]:
person_hh = pd.merge(person,hh[['hhno','hhtaz']],on='hhno',how='left')
person_hh = pd.merge(person_hh, rgc_taz, left_on='hhtaz', right_on='taz', how='left')
person_hh = pd.merge(person_hh, special_taz, left_on='hhtaz', right_on='TAZ', how='left')
df = pd.merge(driver_trips,person_hh,on=['hhno','pno'], how='left')

In [13]:
reg_vmt = df['travdist'].sum()/person_hh['psexpfac'].sum()

In [14]:
vmt = pd.DataFrame(df[['Minority','travdist']].groupby('Minority').sum()['travdist'])
# Population totals
pop = person_hh[['Minority','psexpfac']].groupby(['Minority']).sum()
minority_vmt = pd.DataFrame(vmt['travdist']/pop['psexpfac']).loc[1].values[0]

In [15]:
vmt = pd.DataFrame(df[['Low Income','travdist']].groupby('Low Income').sum()['travdist'])
# Population totals
pop = person_hh[['Low Income','psexpfac']].groupby(['Low Income']).sum()
lowinc_vmt = pd.DataFrame(vmt['travdist']/pop['psexpfac']).loc[1].values[0]

In [16]:
_df = pd.DataFrame([reg_vmt,minority_vmt,lowinc_vmt])
_df.columns = ['Average VMT per Capita']
_df.index = ['Region','People of Color','People of Lower Income']
_df

,Average VMT per Capita
Region,14.51
People of Color,12.78
People of Lower Income,12.43
